In [1]:
import sys
sys.path.append('./../')
import tensorflow as tf
print(tf.__version__)

# Helper libraries
from tensorflow import keras
from keras import mixed_precision
import numpy as np
import time
import maxine
import sys
from maxine.layers import SoftMaxMin_Pruned_v02
import keras.backend as k
import pruning_tool_v2
from maxine.tools import pruning_tool_MLP_v3
from maxine.tools import pruning_tool_MLP_v4_1
import tensorflow_model_optimization as tfmot
from keras import regularizers
from keras.callbacks import EarlyStopping
from scipy.io import loadmat

2023-12-18 11:11:33.854434: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-18 11:11:34.618377: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.0


In [2]:
# LOAD DATA
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
rows, cols = 28, 28
x_train = x_train.reshape(x_train.shape[0], rows, cols, 1)
x_test = x_test.reshape(x_test.shape[0], rows, cols, 1)
input_shape = (rows, cols, 1)

# convert to float and normalize
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

val_labels = y_train[-5000:]
train_labels = y_train[:-5000]
test_labels = y_test

# one-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
# Validation data
x_val = x_train[-5000:]
y_val = y_train[-5000:]

x_train = x_train[:-5000]
y_train = y_train[:-5000]

In [3]:
lenet5 = tf.keras.models.load_model('./models/paper/lenet5')

2023-12-18 11:11:38.832489: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2031] Ignoring visible gpu device (device: 2, name: NVIDIA GeForce GT 1030, pci bus id: 0000:84:00.0, compute capability: 6.1) with core count: 3. The minimum required count is 8. You can adjust this requirement with the env var TF_MIN_GPU_MULTIPROCESSOR_COUNT.
2023-12-18 11:11:40.156667: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 23664 MB memory:  -> device: 0, name: Tesla P40, pci bus id: 0000:03:00.0, compute capability: 6.1
2023-12-18 11:11:40.157384: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 23664 MB memory:  -> device: 1, name: Tesla P40, pci bus id: 0000:04:00.0, compute capability: 6.1


In [4]:
lenet5.evaluate(x_test, y_test)

2023-12-18 11:11:41.686660: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902


 36/313 [==>...........................] - ETA: 0s - loss: 0.3179 - accuracy: 0.8898

2023-12-18 11:11:42.199705: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


313/313 [==============================] - 2s 3ms/step - loss: 0.3291 - accuracy: 0.8897


[0.3290984034538269, 0.8896999955177307]

In [5]:
lenet5.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 24, 24, 6)         156       
                                                                 
 average_pooling2d_6 (Avera  (None, 12, 12, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_8 (Conv2D)           (None, 8, 8, 16)          2416      
                                                                 
 average_pooling2d_7 (Avera  (None, 4, 4, 16)          0         
 gePooling2D)                                                    
                                                                 
 flatten_6 (Flatten)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 120)              

In [6]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [7]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0, weight_decay=0)
lenet5.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])

In [7]:
lenet5.get_config()

{'name': 'sequential_4',
 'layers': [{'module': 'keras.layers',
   'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 28, 28, 1),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'conv2d_7_input'},
   'registered_name': None},
  {'module': 'keras.layers',
   'class_name': 'Conv2D',
   'config': {'name': 'conv2d_7',
    'trainable': True,
    'dtype': 'float32',
    'batch_input_shape': (None, 28, 28, 1),
    'filters': 6,
    'kernel_size': (5, 5),
    'strides': (1, 1),
    'padding': 'valid',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'groups': 1,
    'activation': 'tanh',
    'use_bias': True,
    'kernel_initializer': {'module': 'keras.initializers',
     'class_name': 'GlorotUniform',
     'config': {'seed': None},
     'registered_name': None},
    'bias_initializer': {'module': 'keras.initializers',
     'class_name': 'Zeros',
     'config': {},
     'registered_name': None},
    'kernel_regularizer': None

p=0

In [8]:
lenet5_pruned, lenet5_hp1, lenet5_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(lenet5, x_train, train_labels, x_val, val_labels, x_test, test_labels, batch_size=128, morph_epochs=20, mlp_epochs=10, mlp_epochs_refit=30, p=0, optimizer_config=tf.optimizers.serialize(optimizer), extreme=False)

Training morphological model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 120)              30720     
                                                                 
 dense_1 (Dense)             (55000, 10)               1200      
                                                                 
Total params: 31920 (124.69 KB)
Trainable params: 31920 (124.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
1719/1719 - 10s - loss: 0.9091 - accuracy: 0.7362 - val_loss: 0.4886 - val_accuracy: 0.8276 - 10s/epoch - 6ms/step
Epoch 2/20
1719/1719 - 8s - loss: 0.4471 - accuracy: 0.8419 - val_loss: 0.4139 - val_accuracy: 0.8470 - 8s/epoch - 5ms/step
Epoch 3/20
1719/1719 - 8s - loss: 0.3886 - accuracy: 0.8615 - val_loss: 0.3871 - val_accuracy: 0.8592 - 8s/epoch - 5ms/step
Epoch 

In [12]:
w0 = lenet5_pruned.layers[-4].get_weights()[0]
w1 = lenet5_pruned.layers[-2].get_weights()[0]
params_prunedModel = lenet5.count_params() - (w0.size - np.count_nonzero(w0) + w1.size - np.count_nonzero(w1))
print("Parámetros mdoelo sin prunar: " + str(lenet5.count_params()))
print("Parámetros mdoelo prunado: " + str(params_prunedModel))
print("Pesos restantes: " + str(params_prunedModel/lenet5.count_params()*100))

Parámetros mdoelo sin prunar: 44426
Parámetros mdoelo prunado: 9365
Pesos restantes: 21.07999819925269


p=30

In [12]:
lenet5_pruned, lenet5_hp1, lenet5_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(lenet5, x_train, train_labels, x_val, val_labels, x_test, test_labels, batch_size=128, morph_epochs=20, mlp_epochs=10, mlp_epochs_refit=30, p=30, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True)

Training morphological model...
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min_1 (SoftMaxMin  (55000, 120)              30720     
 )                                                               
                                                                 
 dense_2 (Dense)             (55000, 10)               1200      
                                                                 
Total params: 31920 (124.69 KB)
Trainable params: 31920 (124.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
1719/1719 - 9s - loss: 0.8775 - accuracy: 0.7454 - val_loss: 0.4490 - val_accuracy: 0.8390 - 9s/epoch - 5ms/step
Epoch 2/20
1719/1719 - 7s - loss: 0.4063 - accuracy: 0.8569 - val_loss: 0.3741 - val_accuracy: 0.8626 - 7s/epoch - 4ms/step
Epoch 3/20
1719/1719 - 7s - loss: 0.3535 - accuracy: 0.8746 - val_

In [9]:
w0 = lenet5_pruned.layers[-4].get_weights()[0]
w1 = lenet5_pruned.layers[-2].get_weights()[0]
params_prunedModel = lenet5.count_params() - (w0.size - np.count_nonzero(w0) + w1.size - np.count_nonzero(w1))
print("Parámetros mdoelo sin prunar: " + str(lenet5.count_params()))
print("Parámetros mdoelo prunado: " + str(params_prunedModel))
print("Pesos restantes: " + str(params_prunedModel/lenet5.count_params()*100))

Parámetros mdoelo sin prunar: 44426
Parámetros mdoelo prunado: 3842
Pesos restantes: 8.64808895691712


In [8]:
lenet5 = tf.keras.models.load_model('./models/paper/lenet5')
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(lenet5, x_train, train_labels, x_val, val_labels, x_test, test_labels, batch_size=128, morph_epochs=30, mlp_epochs=5, mlp_epochs_refit=30, p=30, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True)

Training morphological model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 120)              30720     
                                                                 
 dense (Dense)               (55000, 10)               1200      
                                                                 
Total params: 31920 (124.69 KB)
Trainable params: 31920 (124.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30


2023-12-18 11:12:10.133534: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x22969910 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-18 11:12:10.133625: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla P40, Compute Capability 6.1
2023-12-18 11:12:10.133642: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla P40, Compute Capability 6.1
2023-12-18 11:12:10.148536: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-18 11:12:10.335243: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-18 11:12:10.511879: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1719/1719 - 11s - loss: 0.9485 - accuracy: 0.7196 - val_loss: 0.5258 - val_accuracy: 0.8048 - 11s/epoch - 6ms/step
Epoch 2/30
1719/1719 - 7s - loss: 0.4883 - accuracy: 0.8259 - val_loss: 0.4429 - val_accuracy: 0.8324 - 7s/epoch - 4ms/step
Epoch 3/30
1719/1719 - 7s - loss: 0.4228 - accuracy: 0.8486 - val_loss: 0.4123 - val_accuracy: 0.8458 - 7s/epoch - 4ms/step
Epoch 4/30
1719/1719 - 7s - loss: 0.3910 - accuracy: 0.8590 - val_loss: 0.3938 - val_accuracy: 0.8536 - 7s/epoch - 4ms/step
Epoch 5/30
1719/1719 - 7s - loss: 0.3717 - accuracy: 0.8664 - val_loss: 0.3691 - val_accuracy: 0.8642 - 7s/epoch - 4ms/step
Epoch 6/30
1719/1719 - 7s - loss: 0.3576 - accuracy: 0.8710 - val_loss: 0.3683 - val_accuracy: 0.8632 - 7s/epoch - 4ms/step
Epoch 7/30
1719/1719 - 7s - loss: 0.3480 - accuracy: 0.8753 - val_loss: 0.3618 - val_accuracy: 0.8646 - 7s/epoch - 4ms/step
Epoch 8/30
1719/1719 - 7s - loss: 0.3392 - accuracy: 0.8773 - val_loss: 0.3521 - val_accuracy: 0.8690 - 7s/epoch - 4ms/step
Epoch 9/30
1719/1

In [11]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0, weight_decay=0)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])
pruned.fit(x_train, train_labels, validation_data=(x_val, val_labels), epochs=20, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, test_labels)

Epoch 1/20
430/430 [==============================] - 4s 7ms/step - loss: 0.2783 - accuracy: 0.9015 - val_loss: 0.3197 - val_accuracy: 0.8868
Epoch 2/20
430/430 [==============================] - 3s 6ms/step - loss: 0.2783 - accuracy: 0.9020 - val_loss: 0.3193 - val_accuracy: 0.8896
Epoch 3/20
430/430 [==============================] - 3s 6ms/step - loss: 0.2782 - accuracy: 0.9017 - val_loss: 0.3198 - val_accuracy: 0.8870
Epoch 4/20
430/430 [==============================] - 3s 7ms/step - loss: 0.2782 - accuracy: 0.9019 - val_loss: 0.3185 - val_accuracy: 0.8892
Epoch 5/20
430/430 [==============================] - 3s 6ms/step - loss: 0.2780 - accuracy: 0.9023 - val_loss: 0.3191 - val_accuracy: 0.8884
Epoch 6/20
430/430 [==============================] - 2s 6ms/step - loss: 0.2777 - accuracy: 0.9023 - val_loss: 0.3179 - val_accuracy: 0.8884
Epoch 7/20
430/430 [==============================] - 3s 7ms/step - loss: 0.2775 - accuracy: 0.9017 - val_loss: 0.3182 - val_accuracy: 0.8852
Epoch 

[0.32291412353515625, 0.8845000267028809]

In [12]:
pruned.save('./models/lenet5_pruned30')

INFO:tensorflow:Assets written to: ./models/lenet5_pruned30/assets


INFO:tensorflow:Assets written to: ./models/lenet5_pruned30/assets


p=20

In [16]:
lenet5 = tf.keras.models.load_model('./models/paper/lenet5')

In [7]:
lenet5_pruned, lenet5_hp1, lenet5_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(lenet5, x_train, train_labels, x_val, val_labels, x_test, test_labels, batch_size=128, morph_epochs=20, mlp_epochs=30, mlp_epochs_refit=30, p=20, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True)

2023-11-30 11:57:25.994336: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-11-30 11:57:26.518310: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Training morphological model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 120)              30720     
                                                                 
 dense (Dense)               (55000, 10)               1200      
                                                                 
Total params: 31920 (124.69 KB)
Trainable params: 31920 (124.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20


2023-11-30 11:57:28.796029: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x22793af0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-30 11:57:28.796119: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla P40, Compute Capability 6.1
2023-11-30 11:57:28.796135: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla P40, Compute Capability 6.1
2023-11-30 11:57:28.812277: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-30 11:57:28.972320: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-30 11:57:29.154132: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1719/1719 - 10s - loss: 0.9218 - accuracy: 0.7307 - val_loss: 0.5182 - val_accuracy: 0.8062 - 10s/epoch - 6ms/step
Epoch 2/20
1719/1719 - 7s - loss: 0.4842 - accuracy: 0.8252 - val_loss: 0.4456 - val_accuracy: 0.8332 - 7s/epoch - 4ms/step
Epoch 3/20
1719/1719 - 7s - loss: 0.4265 - accuracy: 0.8471 - val_loss: 0.4229 - val_accuracy: 0.8390 - 7s/epoch - 4ms/step
Epoch 4/20
1719/1719 - 7s - loss: 0.3960 - accuracy: 0.8577 - val_loss: 0.3932 - val_accuracy: 0.8532 - 7s/epoch - 4ms/step
Epoch 5/20
1719/1719 - 7s - loss: 0.3761 - accuracy: 0.8653 - val_loss: 0.3861 - val_accuracy: 0.8548 - 7s/epoch - 4ms/step
Epoch 6/20
1719/1719 - 7s - loss: 0.3620 - accuracy: 0.8698 - val_loss: 0.3922 - val_accuracy: 0.8498 - 7s/epoch - 4ms/step
Epoch 7/20
1719/1719 - 7s - loss: 0.3512 - accuracy: 0.8737 - val_loss: 0.3757 - val_accuracy: 0.8576 - 7s/epoch - 4ms/step
Epoch 8/20
1719/1719 - 7s - loss: 0.3429 - accuracy: 0.8764 - val_loss: 0.3660 - val_accuracy: 0.8664 - 7s/epoch - 4ms/step
Epoch 9/20
1719/1

In [8]:
w0 = lenet5_pruned.layers[-4].get_weights()[0]
w1 = lenet5_pruned.layers[-2].get_weights()[0]
params_prunedModel = lenet5.count_params() - (w0.size - np.count_nonzero(w0) + w1.size - np.count_nonzero(w1))
print("Parámetros mdoelo sin prunar: " + str(lenet5.count_params()))
print("Parámetros mdoelo prunado: " + str(params_prunedModel))
print("Pesos restantes: " + str(params_prunedModel/lenet5.count_params()*100))

Parámetros mdoelo sin prunar: 44426
Parámetros mdoelo prunado: 3991
Pesos restantes: 8.983478143429522


In [10]:
lenet5_pruned, lenet5_hp1, lenet5_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(lenet5, x_train, train_labels, x_val, val_labels, x_test, test_labels, batch_size=128, morph_epochs=20, mlp_epochs=30, mlp_epochs_refit=30, p=20, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True)

Training morphological model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 120)              30720     
                                                                 
 dense_1 (Dense)             (55000, 10)               1200      
                                                                 
Total params: 31920 (124.69 KB)
Trainable params: 31920 (124.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
1719/1719 - 9s - loss: 0.9104 - accuracy: 0.7321 - val_loss: 0.4854 - val_accuracy: 0.8220 - 9s/epoch - 5ms/step
Epoch 2/20
1719/1719 - 7s - loss: 0.4476 - accuracy: 0.8385 - val_loss: 0.4166 - val_accuracy: 0.8454 - 7s/epoch - 4ms/step
Epoch 3/20
1719/1719 - 7s - loss: 0.3910 - accuracy: 0.8597 - val_loss: 0.3759 - val_accuracy: 0.8584 - 7s/epoch - 4ms/step
Epoch 4/

In [7]:
lenet5_pruned, lenet5_hp1, lenet5_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(lenet5, x_train, train_labels, x_val, val_labels, x_test, test_labels, batch_size=128, morph_epochs=20, mlp_epochs=5, mlp_epochs_refit=30, p=20, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True)

Training morphological model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 120)              30720     
                                                                 
 dense (Dense)               (55000, 10)               1200      
                                                                 
Total params: 31920 (124.69 KB)
Trainable params: 31920 (124.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20


2023-12-13 13:05:41.449043: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x22850780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-13 13:05:41.449108: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla P40, Compute Capability 6.1
2023-12-13 13:05:41.449124: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla P40, Compute Capability 6.1
2023-12-13 13:05:41.462727: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-13 13:05:41.609941: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-13 13:05:41.787392: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1719/1719 - 10s - loss: 0.9322 - accuracy: 0.7301 - val_loss: 0.5189 - val_accuracy: 0.8134 - 10s/epoch - 6ms/step
Epoch 2/20
1719/1719 - 7s - loss: 0.4808 - accuracy: 0.8273 - val_loss: 0.4305 - val_accuracy: 0.8416 - 7s/epoch - 4ms/step
Epoch 3/20
1719/1719 - 7s - loss: 0.4212 - accuracy: 0.8498 - val_loss: 0.4084 - val_accuracy: 0.8510 - 7s/epoch - 4ms/step
Epoch 4/20
1719/1719 - 7s - loss: 0.3897 - accuracy: 0.8596 - val_loss: 0.3785 - val_accuracy: 0.8588 - 7s/epoch - 4ms/step
Epoch 5/20
1719/1719 - 7s - loss: 0.3699 - accuracy: 0.8679 - val_loss: 0.3714 - val_accuracy: 0.8594 - 7s/epoch - 4ms/step
Epoch 6/20
1719/1719 - 7s - loss: 0.3553 - accuracy: 0.8720 - val_loss: 0.3571 - val_accuracy: 0.8632 - 7s/epoch - 4ms/step
Epoch 7/20
1719/1719 - 7s - loss: 0.3452 - accuracy: 0.8758 - val_loss: 0.3640 - val_accuracy: 0.8652 - 7s/epoch - 4ms/step
Epoch 8/20
1719/1719 - 7s - loss: 0.3374 - accuracy: 0.8779 - val_loss: 0.3584 - val_accuracy: 0.8656 - 7s/epoch - 4ms/step
Epoch 9/20
1719/1

In [9]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0, weight_decay=0)
lenet5_pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])
lenet5_pruned.fit(x_train, train_labels, validation_data=(x_val, val_labels), epochs=40, batch_size=128, callbacks=[es])
lenet5_pruned.evaluate(x_test, test_labels)

Epoch 1/40
430/430 [==============================] - 4s 7ms/step - loss: 0.2642 - accuracy: 0.9055 - val_loss: 0.2993 - val_accuracy: 0.8938
Epoch 2/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2641 - accuracy: 0.9055 - val_loss: 0.2994 - val_accuracy: 0.8938
Epoch 3/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2641 - accuracy: 0.9057 - val_loss: 0.2994 - val_accuracy: 0.8936
Epoch 4/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2640 - accuracy: 0.9057 - val_loss: 0.2994 - val_accuracy: 0.8946
Epoch 5/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2640 - accuracy: 0.9056 - val_loss: 0.2993 - val_accuracy: 0.8948
Epoch 6/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2640 - accuracy: 0.9058 - val_loss: 0.2994 - val_accuracy: 0.8942
Epoch 7/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2640 - accuracy: 0.9055 - val_loss: 0.2992 - val_accuracy: 0.8942
Epoch 

[0.31338709592819214, 0.8891000151634216]

p=10

In [20]:
lenet5 = tf.keras.models.load_model('./models/paper/lenet5')

In [21]:
pruned, lenet5_hp1, lenet5_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(lenet5, x_train, train_labels, x_val, val_labels, x_test, test_labels, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=30, p=10, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True)

Training morphological model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 120)              30720     
                                                                 
 dense_1 (Dense)             (55000, 10)               1200      
                                                                 
Total params: 31920 (124.69 KB)
Trainable params: 31920 (124.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1719/1719 - 9s - loss: 0.9385 - accuracy: 0.7252 - val_loss: 0.5290 - val_accuracy: 0.8086 - 9s/epoch - 5ms/step
Epoch 2/30
1719/1719 - 7s - loss: 0.4905 - accuracy: 0.8247 - val_loss: 0.4457 - val_accuracy: 0.8374 - 7s/epoch - 4ms/step
Epoch 3/30
1719/1719 - 7s - loss: 0.4293 - accuracy: 0.8455 - val_loss: 0.4227 - val_accuracy: 0.8480 - 7s/epoch - 4ms/step
Epoch 4/

In [13]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0, weight_decay=0)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])
pruned.fit(x_train, train_labels, validation_data=(x_val, val_labels), epochs=10, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, test_labels)

Epoch 1/40
430/430 [==============================] - 5s 7ms/step - loss: 0.2554 - accuracy: 0.9085 - val_loss: 0.2904 - val_accuracy: 0.8936
Epoch 2/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2544 - accuracy: 0.9088 - val_loss: 0.2906 - val_accuracy: 0.8956
Epoch 3/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2543 - accuracy: 0.9086 - val_loss: 0.2910 - val_accuracy: 0.8948
Epoch 4/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2540 - accuracy: 0.9089 - val_loss: 0.2905 - val_accuracy: 0.8948
Epoch 5/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2540 - accuracy: 0.9085 - val_loss: 0.2901 - val_accuracy: 0.8942
Epoch 6/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2538 - accuracy: 0.9090 - val_loss: 0.2898 - val_accuracy: 0.8950
Epoch 7/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2535 - accuracy: 0.9089 - val_loss: 0.2910 - val_accuracy: 0.8938
Epoch 

[0.3042330741882324, 0.8919000029563904]

In [14]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0, weight_decay=0)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])
pruned.fit(x_train, train_labels, validation_data=(x_val, val_labels), epochs=10, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, test_labels)

Epoch 1/10
430/430 [==============================] - 4s 7ms/step - loss: 0.2498 - accuracy: 0.9102 - val_loss: 0.2894 - val_accuracy: 0.8954
Epoch 2/10
430/430 [==============================] - 3s 7ms/step - loss: 0.2496 - accuracy: 0.9104 - val_loss: 0.2893 - val_accuracy: 0.8960
Epoch 3/10
430/430 [==============================] - 3s 7ms/step - loss: 0.2496 - accuracy: 0.9104 - val_loss: 0.2891 - val_accuracy: 0.8958
Epoch 4/10
430/430 [==============================] - 3s 7ms/step - loss: 0.2495 - accuracy: 0.9107 - val_loss: 0.2891 - val_accuracy: 0.8960
Epoch 5/10
430/430 [==============================] - 3s 7ms/step - loss: 0.2495 - accuracy: 0.9104 - val_loss: 0.2892 - val_accuracy: 0.8966
Epoch 6/10
430/430 [==============================] - 3s 7ms/step - loss: 0.2495 - accuracy: 0.9105 - val_loss: 0.2892 - val_accuracy: 0.8966
Epoch 7/10
430/430 [==============================] - 3s 7ms/step - loss: 0.2495 - accuracy: 0.9101 - val_loss: 0.2891 - val_accuracy: 0.8958
Epoch 

[0.3032292425632477, 0.891700029373169]

In [15]:
w0 = pruned.layers[-4].get_weights()[0]
w1 = pruned.layers[-2].get_weights()[0]
params_prunedModel = pruned.count_params() - (w0.size - np.count_nonzero(w0) + w1.size - np.count_nonzero(w1))
print("Parámetros mdoelo sin prunar: " + str(pruned.count_params()))
print("Parámetros mdoelo prunado: " + str(params_prunedModel))
print("Pesos restantes: " + str(params_prunedModel/pruned.count_params()*100))

Parámetros mdoelo sin prunar: 44426
Parámetros mdoelo prunado: 4258
Pesos restantes: 9.584477558186649


p = 5

In [22]:
lenet5 = tf.keras.models.load_model('./models/paper/lenet5')
pruned, lenet5_hp1, lenet5_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(lenet5, x_train, train_labels, x_val, val_labels, x_test, test_labels, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=5, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True)

Training morphological model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 120)              30720     
                                                                 
 dense_1 (Dense)             (55000, 10)               1200      
                                                                 
Total params: 31920 (124.69 KB)
Trainable params: 31920 (124.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1719/1719 - 9s - loss: 0.9209 - accuracy: 0.7301 - val_loss: 0.5130 - val_accuracy: 0.8092 - 9s/epoch - 6ms/step
Epoch 2/30
1719/1719 - 7s - loss: 0.4773 - accuracy: 0.8298 - val_loss: 0.4447 - val_accuracy: 0.8352 - 7s/epoch - 4ms/step
Epoch 3/30
1719/1719 - 7s - loss: 0.4218 - accuracy: 0.8483 - val_loss: 0.4169 - val_accuracy: 0.8468 - 7s/epoch - 4ms/step
Epoch 4/

In [24]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0, weight_decay=0)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])
pruned.fit(x_train, train_labels, validation_data=(x_val, val_labels), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, test_labels)

Epoch 1/40
430/430 [==============================] - 4s 7ms/step - loss: 0.2252 - accuracy: 0.9197 - val_loss: 0.2778 - val_accuracy: 0.8986
Epoch 2/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2241 - accuracy: 0.9196 - val_loss: 0.2798 - val_accuracy: 0.8988
Epoch 3/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2239 - accuracy: 0.9197 - val_loss: 0.2776 - val_accuracy: 0.8990
Epoch 4/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2238 - accuracy: 0.9197 - val_loss: 0.2781 - val_accuracy: 0.8982
Epoch 5/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2235 - accuracy: 0.9203 - val_loss: 0.2800 - val_accuracy: 0.8980
Epoch 6/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2236 - accuracy: 0.9205 - val_loss: 0.2782 - val_accuracy: 0.8990
Epoch 7/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2234 - accuracy: 0.9204 - val_loss: 0.2780 - val_accuracy: 0.8976
Epoch 

[0.2923572361469269, 0.8948000073432922]

p = 3

In [26]:
lenet5 = tf.keras.models.load_model('./models/paper/lenet5')
pruned, lenet5_hp1, lenet5_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(lenet5, x_train, train_labels, x_val, val_labels, x_test, test_labels, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=3, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True)

Training morphological model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 120)              30720     
                                                                 
 dense_1 (Dense)             (55000, 10)               1200      
                                                                 
Total params: 31920 (124.69 KB)
Trainable params: 31920 (124.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1719/1719 - 9s - loss: 0.9481 - accuracy: 0.7153 - val_loss: 0.5330 - val_accuracy: 0.8042 - 9s/epoch - 5ms/step
Epoch 2/30
1719/1719 - 7s - loss: 0.4934 - accuracy: 0.8233 - val_loss: 0.4530 - val_accuracy: 0.8284 - 7s/epoch - 4ms/step
Epoch 3/30
1719/1719 - 7s - loss: 0.4292 - accuracy: 0.8463 - val_loss: 0.4110 - val_accuracy: 0.8454 - 7s/epoch - 4ms/step
Epoch 4/

In [28]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0, weight_decay=0)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])
pruned.fit(x_train, train_labels, validation_data=(x_val, val_labels), epochs=20, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, test_labels)

Epoch 1/20
430/430 [==============================] - 4s 7ms/step - loss: 0.2035 - accuracy: 0.9263 - val_loss: 0.2685 - val_accuracy: 0.9060
Epoch 2/20
430/430 [==============================] - 3s 7ms/step - loss: 0.2022 - accuracy: 0.9269 - val_loss: 0.2690 - val_accuracy: 0.9042
Epoch 3/20
430/430 [==============================] - 3s 7ms/step - loss: 0.2020 - accuracy: 0.9273 - val_loss: 0.2688 - val_accuracy: 0.9042
Epoch 4/20
430/430 [==============================] - 3s 7ms/step - loss: 0.2019 - accuracy: 0.9272 - val_loss: 0.2700 - val_accuracy: 0.9040
Epoch 5/20
430/430 [==============================] - 3s 7ms/step - loss: 0.2017 - accuracy: 0.9272 - val_loss: 0.2697 - val_accuracy: 0.9044
Epoch 6/20
430/430 [==============================] - 3s 7ms/step - loss: 0.2015 - accuracy: 0.9276 - val_loss: 0.2696 - val_accuracy: 0.9048
Epoch 7/20
430/430 [==============================] - 3s 7ms/step - loss: 0.2016 - accuracy: 0.9274 - val_loss: 0.2684 - val_accuracy: 0.9026
Epoch 

[0.29501157999038696, 0.896399974822998]

p = 1

In [29]:
lenet5 = tf.keras.models.load_model('./models/paper/lenet5')
pruned, lenet5_hp1, lenet5_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(lenet5, x_train, train_labels, x_val, val_labels, x_test, test_labels, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=1, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True)

Training morphological model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 120)              30720     
                                                                 
 dense_1 (Dense)             (55000, 10)               1200      
                                                                 
Total params: 31920 (124.69 KB)
Trainable params: 31920 (124.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1719/1719 - 10s - loss: 0.9416 - accuracy: 0.7243 - val_loss: 0.5298 - val_accuracy: 0.8086 - 10s/epoch - 6ms/step
Epoch 2/30
1719/1719 - 7s - loss: 0.4823 - accuracy: 0.8293 - val_loss: 0.4367 - val_accuracy: 0.8382 - 7s/epoch - 4ms/step
Epoch 3/30
1719/1719 - 7s - loss: 0.4208 - accuracy: 0.8505 - val_loss: 0.4059 - val_accuracy: 0.8474 - 7s/epoch - 4ms/step
Epoch 

In [31]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0, weight_decay=0)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])
pruned.fit(x_train, train_labels, validation_data=(x_val, val_labels), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, test_labels)

Epoch 1/40
430/430 [==============================] - 4s 7ms/step - loss: 0.2308 - accuracy: 0.9162 - val_loss: 0.2692 - val_accuracy: 0.9018
Epoch 2/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2299 - accuracy: 0.9169 - val_loss: 0.2689 - val_accuracy: 0.9018
Epoch 3/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2297 - accuracy: 0.9164 - val_loss: 0.2683 - val_accuracy: 0.9020
Epoch 4/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2294 - accuracy: 0.9172 - val_loss: 0.2685 - val_accuracy: 0.9026
Epoch 5/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2291 - accuracy: 0.9172 - val_loss: 0.2697 - val_accuracy: 0.9018
Epoch 6/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2290 - accuracy: 0.9169 - val_loss: 0.2693 - val_accuracy: 0.9016
Epoch 7/40
430/430 [==============================] - 3s 7ms/step - loss: 0.2289 - accuracy: 0.9168 - val_loss: 0.2693 - val_accuracy: 0.9024
Epoch 

[0.29613161087036133, 0.8952000141143799]

: 

In [7]:
lenet5.count_params()

44426